## Version Information

<table align="left">
  <thead>
    <tr>
      <th>Component</th>
      <th>Version</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>python</td>
      <td>3.11.5</td>
    </tr>
    <tr>
      <td>cirq</td>
      <td>1.3.0</td>
    </tr>
  </tbody>
</table>

</table>


In [1]:
import cirq

In [2]:
# Pick a qubit.
qubit = cirq.GridQubit(0, 0)

# Create a circuit that applies a square root of NOT gate, then measures the qubit.
circuit = cirq.Circuit(cirq.X(qubit) ** 0.5, cirq.measure(qubit, key='m'))
print("Circuit:")
print(circuit)

# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print("Results:")
print(result)

Circuit:
(0, 0): ───X^0.5───M('m')───
Results:
m=10111000111000011011


In [12]:
import numpy as np
import cirq
from math import gcd, log2, ceil

def find_all_as(N):
    return [n for n in range(2, N) if gcd(n, N) == 1]

class ControlledModExpGate(cirq.Gate):
    def __init__(self, a, N, power):
        super(ControlledModExpGate, self).__init__()
        self.a = a
        self.N = N
        self.power = power
        self.n_len = ceil(log2(N))
        self.ns = find_all_as(N)
        
        if self.a not in self.ns:
            raise ValueError(f"'a' must be one of {self.ns}")
        
    def num_qubits(self):
        return self.n_len + 1  # One control qubit and n_len target qubits

    def _decompose_(self, qubits):
        control = qubits[0]
        targets = qubits[1:]
        
        # Create the circuit for U gate
        U = cirq.Circuit()
        for i, bit in enumerate(format(pow(self.a, self.power, self.N) ^ 1, f'0{self.n_len}b')):
            if bit == '1':
                U.append(cirq.X(targets[i]))
        
        # Convert the U circuit to a gate and control it
        U_gate = U.to_gate()
        controlled_U_gate = U_gate.controlled()
        
        # Apply the controlled U gate to the qubits
        yield controlled_U_gate.on(control, *targets)

    def __str__(self):
        return f"ControlledModExpGate(a={self.a}, N={self.N}, power={self.power})"

# Example parameters
N = 15
a = 2
n_len = ceil(log2(N))
m_len = 2 * n_len

# Create the circuit
qubits = cirq.LineQubit.range(m_len + n_len)
circuit = cirq.Circuit()

# Apply Hadamard gates to the first m_len qubits
circuit.append(cirq.H.on_each(qubits[:m_len]))

# Initialize the auxiliary qubit
circuit.append(cirq.X(qubits[m_len]))

# Apply a sequence of ControlledModExpGate gates
for q in range(m_len):
    power = 2**q
    mod_exp_gate = ControlledModExpGate(a, N, power)
    circuit.append(mod_exp_gate.on(qubits[q], *qubits[m_len:]))

# Add inverse QFT
circuit.append(cirq.inverse(cirq.qft(*qubits[:m_len])))

# Add measurements
circuit.append(cirq.measure(*qubits[:m_len], key='result'))

# Print the circuit
print(circuit)

# Simulate the circuit
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1000)
counts = result.histogram(key='result')

# Convert counts to the same format as Qiskit
counts = {bin(k)[2:].zfill(m_len): v for k, v in counts.items()}

# Plot histogram
keys = list(counts.keys())
values = list(counts.values())
plt.bar(keys, values)
plt.xlabel('State')
plt.ylabel('Counts')
plt.show()

# Compute mean and standard deviation
mean = np.mean(values)
std_dev = np.std(values)

# Define a threshold for outliers
threshold = 3 * std_dev

# Identify outliers
outliers = {key: val for key, val in counts.items() if abs(val - mean) > threshold}

if outliers:
    counts = outliers

# Filter counts to include only those that exceed the threshold
filtered_counts = {key: val for key, val in counts.items() if abs(val - mean) > threshold}

# Plot histogram with horizontal lines for mean and std_dev
fig, ax = plt.subplots(figsize=(17, 5))
ax.bar(filtered_counts.keys(), filtered_counts.values())

# Add horizontal lines
ax.axhline(mean, color='orange', linestyle='--', linewidth=1, label=f'Mean: {mean:.2f}')
ax.axhline(mean + std_dev, color='blue', linestyle='--', linewidth=1, label=f'Std Dev: {std_dev:.2f}')
ax.axhline(mean - std_dev, color='blue', linestyle='--', linewidth=1)
ax.legend()

# Remove x-axis labels
ax.set_xticklabels([])

plt.show()


0: ────H───ControlledModExpGate(a=2, N=15, power=1)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────qft^-1───M('result')───
           │                                                                                                                                                                                                                                                                                                                                                            │        │
1: ────H───┼──────────────────────────────────────────ControlledModExpGate(a=2, N=15, power=2)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

AttributeError: 'Circuit' object has no attribute 'to_gate'

In [34]:
import cirq

# Create qubits
target_qubit = cirq.LineQubit(1)

# Define the circuit
circuit = cirq.Circuit(
    cirq.X(target_qubit),
    cirq.H(target_qubit)
)

# Define a gate class from the circuit
class CustomGate(cirq.Gate):
    def __init__(self, circuit):
        super(CustomGate, self)
        self.circuit = circuit

    def num_qubits(self):
        return len(self.circuit.all_qubits())

    def _decompose_(self, qubits):
        # Map the circuit's qubits to the given qubits
        qubit_map = {q: qubits[i] for i, q in enumerate(self.circuit.all_qubits())}
        return cirq.CircuitOperation(self.circuit.transform_qubits(qubit_map).freeze()).with_qubits(*qubits)

# Instantiate the custom gate
custom_gate = CustomGate(circuit)

# Create the control qubit and target qubit
control_qubit = cirq.LineQubit(0)
target_qubit = cirq.LineQubit(1)

# Create a ControlledGate
controlled_custom_gate = custom_gate.on(target_qubit).controlled_by(control_qubit)

# Create a new circuit and add the ControlledGate
new_circuit = cirq.Circuit()
new_circuit.append(controlled_custom_gate)

print("New Circuit with Controlled Gate:")
print(new_circuit)


New Circuit with Controlled Gate:
0: ───C<__main__.CustomGate object at 0x0000025CBE2A4750>───
      │
1: ───#2────────────────────────────────────────────────────


In [36]:
for i, q in enumerate(circuit.all_qubits()):
    print(i,type(i))
    print(q,type(q))

0 <class 'int'>
q(1) <class 'cirq.devices.line_qubit.LineQubit'>
